In [1]:
import os
new_working_directory = "C:\\Users\\ianja\\REPOS\\Pepperpot\\"
os.chdir(new_working_directory)

In [12]:
# SEGMENT A DIR BY PARAM VARIATION
import os
import shutil
import re

class ParameterValues:
    def __init__(self, epsnx, alfax, betax, epsny, alfay, betay, epsnz, alfaz, betaz):
        self.epsnx = epsnx
        self.alfax = alfax
        self.betax = betax
        self.epsny = epsny
        self.alfay = alfay
        self.betay = betay
        self.epsnz = epsnz
        self.alfaz = alfaz
        self.betaz = betaz

# Specify the local directory containing the images
local_directory = 'local-images-before-after-propagation-bw-shorter'

# Check the current working directory
print(f"Current working directory: {os.getcwd()}")

# Check if the local_directory exists in the current working directory
if not os.path.exists(local_directory):
    print(f"The directory '{local_directory}' does not exist in the current working directory.")
else:
    print(f"The directory '{local_directory}' exists.")

# Create the target directories for organizing the images
target_directory_2param = os.path.join(local_directory, '2-param')
target_directory_3param = os.path.join(local_directory, '3-param')
target_directory_6param = os.path.join(local_directory, '6-param')

# Create the target directories if they don't exist
os.makedirs(target_directory_2param, exist_ok=True)
os.makedirs(target_directory_3param, exist_ok=True)
os.makedirs(target_directory_6param, exist_ok=True)

# Regular expression pattern for extracting parameters from filename
pattern = r"epsnx([\d.-]+)_alfax([\d.-]+)_betax([\d.-]+)_epsny([\d.-]+)_alfay([\d.-]+)_betay([\d.-]+)_epsnz([\d.-]+)_alfaz([\d.-]+)_betaz([\d.-]+)_*[\d]*\.png"

# Iterate over the images in the local directory
for filename in os.listdir(local_directory):
    if filename.endswith('.png'):
        source_path = os.path.join(local_directory, filename)

        # Extract parameters from filename using regex
        match = re.match(pattern, filename)
        if match:
            params = ParameterValues(*match.groups())
            varied_params = sum(param != '0.0' for param in vars(params).values())

            # Move the file to the corresponding target directory based on the number of varied parameters
            if filename.startswith("epsnx0.1_"):
                target_path = os.path.join(target_directory_2param, filename)
            elif params.alfay == "-0.55" and params.betay == "170.0":
                target_path = os.path.join(target_directory_3param, filename)
            else:
                target_path = os.path.join(target_directory_6param, filename)

            shutil.move(source_path, target_path)

print("Image organization completed.")


Current working directory: c:\Users\ianja\REPOS\Pepperpot
The directory 'local-images-before-after-propagation-bw-shorter' exists.
Image organization completed.


In [ ]:
# TEST/TRAIN SPLIT
import os
import random
import shutil
import time

random.seed(42)

original_folders = ["local-images-50cm-propagation-bw-200bin-cropped/2-param/", "local-images-50cm-propagation-bw-200bin-cropped/3-param/", "local-images-10cm-propagation-bw-200bin-cropped"]
train_folder = "./train"
test_folder = "./test"

os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

train_ratio = 0.8

file_list = []

for folder in original_folders:
    file_list.extend([os.path.join(folder, file) for file in os.listdir(folder)])
    print(len(file_list))

random.shuffle(file_list)

train_size = int(len(file_list) * train_ratio)

train_files = file_list[:train_size]
test_files = file_list[train_size:]

for src_path in train_files:
    file = os.path.basename(src_path)
    dst_path = os.path.join(train_folder, file)

    # Check if a file with the same name already exists in the target directory
    if os.path.exists(dst_path):
        # Split filename into name and extension
        base, ext = os.path.splitext(file)
        # Append a unique identifier to the filename
        new_filename = f"{base}_{random.randint(1000,9999)}{ext}"
        dst_path = os.path.join(train_folder, new_filename)

    shutil.copy(src_path, dst_path)

for src_path in test_files:
    file = os.path.basename(src_path)
    dst_path = os.path.join(test_folder, file)

    # Check if a file with the same name already exists in the target directory
    if os.path.exists(dst_path):
        # Split filename into name and extension
        base, ext = os.path.splitext(file)
        # Append a unique identifier to the filename
        new_filename = f"{base}_{random.randint(1000,9999)}{ext}"
        dst_path = os.path.join(test_folder, new_filename)

    shutil.copy(src_path, dst_path)

print("Dataset split completed!")


In [22]:
# force clear cache and memory
import torch
torch.cuda.empty_cache()
import gc
gc.collect()

0

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
import re
import cv2
import numpy as np
from torchvision import transforms
import random

# ensure determinism
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [20]:
# 3-PARAMETER SIDE_BY_SIDE MODEL 
NAME = "3param-dbwd"

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 197 * 98, 256)  # Adjusted input size
        self.fc2 = nn.Linear(256, 2) 

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


In [16]:
# DB for DOUBLE WIDE IMAGE
class CustomDataset(Dataset):
    def __init__(self, image_files):
        self.image_files = image_files
        self.pattern = r"epsnx([\d.-]+)_alfax([\d.-]+)_betax([\d.-]+)_epsny([\d.-]+)_alfay([\d.-]+)_betay([\d.-]+)_epsnz([\d.-]+)_alfaz([\d.-]+)_betaz([\d.-]+)_*[\d]*\.png"

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        image_path = self.image_files[index]
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (394, 197)).astype(np.float32) / 255.0

        # add dummy dim (1, x, x)
        image = np.expand_dims(image, axis=0) 

        image_name = os.path.basename(image_path)
        matches = re.search(self.pattern, image_name)
        if matches is None:
            print(f"No match found for file: {image_name}")
            return None
        variables = [float(matches.group(i)) for i in range(1, 10) if matches.group(i)]
        epsnx = variables[0]  # epsnx value
        alfa_x = variables[1]  # alfax value
        return image, torch.tensor([epsnx, alfa_x])  # Returns epsnx, alfa x and betax as the labels


In [17]:
# Load the dataset
image_dirs = ["local-images-before-after-propagation-bw-shorter/2-param/", "local-images-before-after-propagation-bw-shorter/3-param/"]
image_files = [os.path.join(dir_path, name) 
               for dir_path in image_dirs 
               for name in sorted(os.listdir(dir_path))]

total_size = len(image_files)
train_prop = 0.90
val_prop = 0.10

train_size = int(train_prop * total_size)
val_size = int(val_prop * total_size)
test_size = total_size - train_size - val_size
print(train_size, val_size, test_size)

# Split the dataset
train_files = image_files[:train_size]
val_files = image_files[train_size:train_size + val_size]
test_files = image_files[train_size + val_size:]

train_dataset = CustomDataset(train_files)
val_dataset = CustomDataset(val_files)
test_dataset = CustomDataset(test_files)

3799 422 1


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [23]:
batch_size = 4
learning_rate = 0.001
num_epochs = 30

model = CNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

total_steps = len(train_dataloader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_steps}], Loss: {loss.item():.4f}")
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")
# filename = f"cnn_{NAME}_size{total_size}_train{train_prop}_val{val_prop}_bs{batch_size}_lr{learning_rate}_e{num_epochs}.pth"
filename = f"cnn_{NAME}_bs{batch_size}_lr{learning_rate}_e{num_epochs}_trn{train_size}.pth"
torch.save(model.state_dict(), filename)

Epoch [1/30], Loss: 3.5521
Epoch [2/30], Loss: 4.2344
Epoch [3/30], Loss: 8.2496
Epoch [4/30], Loss: 4.2512
Epoch [5/30], Loss: 4.0006
Epoch [6/30], Loss: 6.4638
Epoch [7/30], Loss: 3.4164
Epoch [8/30], Loss: 4.2968
Epoch [9/30], Loss: 3.2311
Epoch [10/30], Loss: 3.0185
Epoch [11/30], Loss: 3.2910
Epoch [12/30], Loss: 4.2347
Epoch [13/30], Loss: 8.8917
Epoch [14/30], Loss: 3.4224
Epoch [15/30], Loss: 0.4820
Epoch [16/30], Loss: 8.2682
Epoch [17/30], Loss: 6.6173
Epoch [18/30], Loss: 5.1701
Epoch [19/30], Loss: 6.9315
Epoch [20/30], Loss: 2.0271
Epoch [21/30], Loss: 2.5312
Epoch [22/30], Loss: 7.8179
Epoch [23/30], Loss: 1.6436
Epoch [24/30], Loss: 2.0422
Epoch [25/30], Loss: 7.0844
Epoch [26/30], Loss: 6.5665
Epoch [27/30], Loss: 2.6208
Epoch [28/30], Loss: 3.8862
Epoch [29/30], Loss: 7.5379
Epoch [30/30], Loss: 2.5882


In [ ]:
# with simple validation circuit

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

best_loss = np.inf
saved_models = []
for epoch in range(num_epochs):
    # Training loop
    model.train()
    for i, (images, labels) in enumerate(train_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch + 1}/{num_epochs}], Training Loss: {loss.item():.4f}")
    
    # Validation loop
    model.eval() 
    with torch.no_grad():
        for i, (images, labels) in enumerate(val_dataloader): 
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            val_loss = criterion(outputs, labels)

    print(f"Epoch [{epoch + 1}/{num_epochs}], Validation Loss: {val_loss.item():.4f}")

    # Check if this model is better than previous ones
    if val_loss < best_loss or epoch % 10 == 0:  # save model every 10 epochs and if it performs better
        best_loss = min(best_loss, val_loss)
        filename = f"cnn_{NAME}_bs{batch_size}_lr{learning_rate}_e{epoch+1}_trn{train_size}.pth"
        torch.save(model.state_dict(), filename)
        saved_models.append(filename)
        # Keep the best 2 models and delete the rest
        if len(saved_models) > 2:
            os.remove(saved_models.pop(0))  # delete oldest model

print(f"Best Validation Loss: {best_loss:.4f}")


In [ ]:
# with validation function (works worse)
from sklearn.model_selection import ParameterGrid
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define your parameter grid
param_grid = {
    'batch_size': [4],
    'learning_rate': [0.001],
    'num_epochs': [300]
}

grid = ParameterGrid(param_grid)

for params in grid:
    batch_size = params['batch_size']
    learning_rate = params['learning_rate']
    num_epochs = params['num_epochs']

    filename = f"cnn_{NAME}_size{total_size}_train{train_prop}_val{val_prop}_bs{batch_size}_lr{learning_rate}_e{num_epochs}.pth"

    # If the model file already exists, skip this iteration
    if os.path.isfile(filename):
        print(f"Model file {filename} already exists. Skipping...")
        continue

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    model = CNN().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model
    losses = []
    val_losses = []
    avg_loss_10_epochs = 0.0
    avg_val_loss_10_epochs = 0.0

    for epoch in range(num_epochs):
        # Training
        model.train()  # Set the model to training mode

        for images, labels in train_dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        losses.append(loss.item())
        avg_loss_10_epochs += loss.item()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {loss.item():.4f}")

        # Validation
        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():
            total_val_loss = 0
            for val_images, val_labels in val_dataloader:
                val_images = val_images.to(device)
                val_labels = val_labels.to(device)
                val_outputs = model(val_images)
                val_loss = criterion(val_outputs, val_labels)
                total_val_loss += val_loss.item()
                val_losses.append(val_loss.item())

            print(f"Val Loss: {val_loss.item():.4f}")       
        
        # Print average training loss across the last 10 epochs
        if (epoch + 1) % 10 == 0:
            avg_loss_last_10_epochs = sum(losses[-10:]) / 10
            print(f"Epoch [{epoch + 1}/{num_epochs}], Average Train Loss (Last 10 epochs): {avg_loss_last_10_epochs:.4f}")
            avg_loss_10_epochs = 0.0

        # Print average training loss across all epochs every 10 epochs
        if (epoch + 1) % 10 == 0:
            avg_loss_all_epochs = sum(losses) / len(losses)
            print(f"Epoch [{epoch + 1}/{num_epochs}], Average Train Loss (All epochs): {avg_loss_all_epochs:.4f}")

    # Test
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        total_test_loss = 0
        for test_images, test_labels in test_dataloader:
            test_images = test_images.to(device)
            test_labels = test_labels.to(device)
            test_outputs = model(test_images)
            test_loss = criterion(test_outputs, test_labels)
            total_test_loss += test_loss.item()
        avg_test_loss = total_test_loss / len(test_dataloader)
    print(f"Test Loss: {avg_test_loss:.4f}")

    torch.save(model.state_dict(), filename)


In [9]:
def evaluate_model(model, test_dataset, num_variables):
    model.eval()
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    total_mae = torch.zeros(num_variables, device=device)
    total_mape = torch.zeros(num_variables, device=device)
    total_smape = torch.zeros(num_variables, device=device)
    total_mse = torch.zeros(num_variables, device=device)
    total_count = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)

            absolute_error = torch.abs(outputs - labels)
            total_mae += absolute_error.sum(dim=0)

            non_zero_mask = torch.abs(labels) > 1e-8
            percentage_error = (absolute_error / torch.abs(labels)) * 100
            total_mape += (percentage_error * non_zero_mask).sum(dim=0)

            smape = 200.0 * torch.abs(outputs - labels) / (torch.abs(outputs) + torch.abs(labels) + torch.finfo(torch.float32).eps)
            total_smape += smape.sum(dim=0)
            
            mse = (outputs - labels) ** 2
            total_mse += mse.sum(dim=0)

            total_count += labels.size(0)

    mae = total_mae / total_count
    mape = total_mape / total_count
    smape = total_smape / total_count
    rmse = torch.sqrt(total_mse / total_count)

    return mae.cpu().numpy(), mape.cpu().numpy(), smape.cpu().numpy(), rmse.cpu().numpy()


In [10]:
# RUN MODEL EVALUATION
import glob
import torch

# Load models and evaluate their performance
model_dir = './'  # Specify ysour directory where models are saved

model_performance = []

# Load each model and evaluate it
for model_file in glob.glob(model_dir + '/*.pth'):
    # Load model
    model = CNN()
    model.load_state_dict(torch.load(model_file))
    model.to(device)

    # Evaluate model
    mae, mape, smape, rmse = evaluate_model(model, test_dataset, 2)
    aggregate_score = np.mean([mae, mape, smape, rmse])
    model_performance.append((model_file, mae, mape, smape, rmse, aggregate_score))

# Sort models based on aggregate score
model_performance.sort(key=lambda x: x[-1])

# Print the performance of each model
for model_info in model_performance:
    model_file, mae, mape, smape, rmse, aggregate_score = model_info
    print(f"Model: {model_file}, Aggregate Score: {aggregate_score:.4f}")
    print(f"Mean Absolute Error: {mae}")
    print(f"Mean Absolute Percentage Error: {mape}")
    print(f"Symmetric Mean Absolute Percentage Error: {smape}")
    print(f"Root Mean Square Error: {rmse}")
    print()

# Open the text document in write mode
with open('.txt', 'w') as f:
    # Print model performance in order
    for model_info in model_performance:
        model_file, mae, mape, smape, rmse, aggregate_score = model_info
        f.write(f"Model: {model_file}, Aggregate Score: {aggregate_score:.4f}\n")
        for i in range(2):
            f.write(f"Metrics for prediction {i+1}:\n")
            f.write(f"Mean Absolute Error: {mae[i]:.4f}\n")
            f.write(f"Mean Absolute Percentage Error: {mape[i]:.2f}%\n")
            f.write(f"Symmetric Mean Absolute Percentage Error: {smape[i]:.2f}%\n")
            f.write(f"Root Mean Square Error: {rmse[i]:.4f}\n")
        f.write("\n")


Model: .\cnn_3param-2xeverything-bw_bs4_lr0.001_e100.pth, Aggregate Score: 20.4264
Mean Absolute Error: [ 0.16377206  0.6010109  12.177272  ]
Mean Absolute Percentage Error: [58.97214   59.230194   5.8334346]
Symmetric Mean Absolute Percentage Error: [42.6713    41.39358    5.8123317]
Root Mean Square Error: [ 0.21665739  0.7982512  17.247149  ]

